In [1]:
import numpy as np
import pandas as pd
from ntf_cython.random import algo42, algo44, admm, rel_error
from ntf_cython.nmf import nmf
from collections import OrderedDict
import scipy

%matplotlib inline

## Synthetic Random Matrix

In [2]:
m = 10000
n = 1000
k = 100
A0 = np.random.randn(m, k)
A1 = np.random.randn(k, n)
A = A0.dot(A1)

## Compression Algorithms

## Direct Method

In [3]:
%time QA, RA = np.linalg.qr(A)

CPU times: user 2.35 s, sys: 118 ms, total: 2.47 s
Wall time: 1.34 s


In [4]:
%time QRA = QA.dot(RA)

CPU times: user 598 ms, sys: 17.1 ms, total: 615 ms
Wall time: 326 ms


In [5]:
rel_error(A, QRA)

9.2598453939395887e-16

## Algo 41. Randomized Range Finder

In [6]:
def algo41(A, r):
    n = A.shape[1]
    W = np.random.rand(n, r)
    Y = A.dot(W)
    Q, R = np.linalg.qr(Y)
    return Q

In [7]:
%time
r = k
Q = algo41(A, r)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


In [8]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 113 ms, sys: 11.1 ms, total: 124 ms
Wall time: 81.7 ms


In [9]:
rel_error(A, QRB)

1.0886781630551195e-14

## Algo42. Adaptive Randomized Range Finder

In [10]:
%time
eps = 0.01
r = 5
Q = algo42(A, eps, r)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


In [11]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 124 ms, sys: 33.7 ms, total: 158 ms
Wall time: 89.6 ms


In [12]:
rel_error(A, QRB)

3.176463570166674e-14

## Algo43. Randomized Power Iteration

In [13]:
def algo43(A, r=100, q=1):
    
    n = A.shape[1]
    W = np.random.normal(size=(n, r))
    Y = ((A.dot(A.T))**q).dot(A).dot(W)
    Q, _ = np.linalg.qr(Y)
    return Q

In [14]:
%%time
q = 1
r = k
Q = algo43(A, r, q)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 10.4 s, sys: 622 ms, total: 11.1 s
Wall time: 6.26 s


In [15]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 114 ms, sys: 17 ms, total: 131 ms
Wall time: 72.5 ms


In [16]:
rel_error(A, QRB)

1.4296963739579134e-14

## Algo44. Randomized Subspace Iteration

In [17]:
%%time
k = 100
q = 1
r = k
Q = algo44(A, q, r)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 503 ms, sys: 21.4 ms, total: 524 ms
Wall time: 272 ms


In [18]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 118 ms, sys: 23.1 ms, total: 142 ms
Wall time: 85.2 ms


In [19]:
rel_error(A, QRB)

1.1150750115892608e-15

## Algo45. Fast Randomized Range Finder

In [38]:
def compute_complex_diagonal(n):
    a = np.random.uniform(size=(n,)) + 1j * np.random.uniform(size=(n,))
    return np.diag(a / np.linalg.norm(a))

In [39]:
def random_permutation_matrix(n, r=None):
    
    R = np.identity(n)
    np.random.shuffle(R)
    if r is not None:
        R = R[:, :r]
    return R

In [46]:
def compute_unaryDFT(n):
    
    q = np.tile(np.arange(0,n),n).reshape(n,n)
    F = (1/np.sqrt(n)) * np.exp((-2 * 1j * np.pi * q.T * q) / n)
    return F

In [47]:
def SRFT(n, r):
    
    D = compute_complex_diagonal(n)
    F = compute_unaryDFT(n)
    R = random_permutation_matrix(n, r)
    return np.sqrt(n / r) * np.linalg.multi_dot([D, F, R])
    

In [48]:
def algo45(A, r):
    
    n = A.shape[1]
    W = SRFT(n, r)
    Y = A.dot(W)
    q, _ = np.linalg.qr(Y)
    
    return q

In [49]:
%%time
k = 100
r = k
Q = algo45(A,r)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 1.27 s, sys: 193 ms, total: 1.47 s
Wall time: 846 ms


In [50]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 453 ms, sys: 82.4 ms, total: 535 ms
Wall time: 293 ms


In [51]:
rel_error(A, QRB)

1.410169063511606

## Algo46: Gaussian Subsampled Random Fourier Transform 

In [28]:
def givens_rotation(n, i, j, theta):
    
    givens_rotation = np.identity(n)
    givens_rotation[i, i] = np.cos(theta)
    givens_rotation[j, j] = np.cos(theta)
    givens_rotation[i, j] = np.sin(theta)
    givens_rotation[j, i] = - np.sin(theta)
    return givens_rotation

In [29]:
def chain_random_givens_rotation(n, r=None):

    P = random_permutation_matrix(n, r)
    for i in range(n - 1):
        P = np.dot(P, givens_rotation(n, i, i + 1, np.random.normal()))
    return P

In [30]:
def GSRFT(n, r, num_iteration):
    
    params = OrderedDict()
    params['D'+str(num_iteration)] = compute_complex_diagonal(n)
    i = num_iteration - 1

    while (i >= 0) and (i <= num_iteration - 1):
        params['theta'+str(i)] = chain_random_givens_rotation(n)
        params['D'+str(i)] = compute_complex_diagonal(n)
        i = i - 1

    F = compute_unaryDFT(n)
    R = random_permutation_matrix(n, r)
    
    a = np.linalg.multi_dot(list(params.values())[:-1])
        
    return np.linalg.multi_dot([a,params['D0'],F,R])
        

In [31]:
def algo46(A, r, num_iteration=3):
    
    n = A.shape[1]
    W = GSRFT(n, r, num_iteration)
    Y = A.dot(W)
    q, _ = np.linalg.qr(Y)
    
    return q

In [32]:
%%time
k = 100
r = k
Q = algo46(A,r,3)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 3min 24s, sys: 9.92 s, total: 3min 34s
Wall time: 1min 48s


In [33]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 390 ms, sys: 77.7 ms, total: 468 ms
Wall time: 258 ms


In [34]:
rel_error(A, QRB)

1.4129892428977826

## Algo: Structured Compression Algorithm

### based on Figure 1 of the book: Compressed Nonnegative Matrix Factorization Is Fast And Accurate

In [35]:
def structured_compression(A, r, q, oversampling):
    
    n = A.shape[1]
    W = np.random.normal(size = (n, r + oversampling))
    Y = ((A.dot(A.T))**q).dot(A).dot(W)
    q, _ = np.linalg.qr(Y)
    
    return q

In [39]:
%%time
oversampling = 5
q = 3
r = 200
Q = structured_compression(A, r, q, oversampling)
B = Q.T.dot(A)
QB, RB = np.linalg.qr(B)

CPU times: user 14.8 s, sys: 832 ms, total: 15.6 s
Wall time: 10.4 s


In [40]:
%time QRB = Q.dot(QB).dot(RB)

CPU times: user 215 ms, sys: 27 ms, total: 242 ms
Wall time: 139 ms


In [41]:
rel_error(A, QRB)

0.22567490928872153

## Algo: Count Gauss Algo: How to fake multiply a matrix

In [211]:
def count_gauss(A, rank, oversampling_factor):
    
    """
    Based on Algorithm 1 of the paper https://arxiv.org/pdf/1606.05732v1.pdf
    """
    
    d = A.shape[0]
    
    k = rank * oversampling_factor
    
    R = np.random.randint(k, size=d)
    C = np.arange(d)
    D = np.random.choice([-1,1], size=d)
    S = scipy.sparse.csc_matrix((D, (R, C)), shape=(k, d))
    
    G = np.random.randn(rank, k)
    #Z = np.linalg.multi_dot([G,S, A])
    
    print(G.shape)
    print(S.T.shape)
    Q = S.T.dot(G.T)
    Z = G.dot(S.dot(A))
    
    return Q,Z
    

In [212]:
A.shape

(10000, 1000)

In [275]:
%%time
oversampling_factor = 10
r = 1000
Q, B = count_gauss(A, r, oversampling_factor)
QB, RB = np.linalg.qr(B)

(1000, 10000)
(10000, 10000)
CPU times: user 1.86 s, sys: 151 ms, total: 2.01 s
Wall time: 1.32 s


In [276]:
QB.shape,RB.shape

((1000, 1000), (1000, 1000))

In [277]:
Q.shape

(10000, 1000)

In [278]:
Q.shape,QB.shape,RB.shape

((10000, 1000), (1000, 1000), (1000, 1000))

In [279]:
QRB = Q.dot(QB).dot(RB)

In [280]:
rel_error(A, QRB)

3462.9824297333712

In [281]:
A.shape

(10000, 1000)

In [100]:
QRB.shape

(10000, 1000)

In [101]:
G.shape,S.shape,A.shape

((100, 500), (500, 10000), (10000, 1000))